<h1>
Pruned Variables, Pruned Values, Aggregated Target (No Separate Morbidities)
</h1>

In [ ]:
from pathlib import Path
import re
import pandas as pd
import numpy as np

BASE_DIR = Path().resolve().parent

<h2>Tidying Up</h2>

In [ ]:
df = pd.read_csv(BASE_DIR / 'natality_7yr_test_data.csv')

df.dtypes

In [ ]:
df['cig_rec'].value_counts(normalize=True)

In [ ]:
df.nunique()

In [ ]:
df['priordead'].value_counts(normalize=True)

In [ ]:
binary_cols = [col for col in df.columns if df[col].nunique() == 2 and df[col].dtype == 'int64']

binary_cols

In [ ]:
df['imp_sex'].unique()

In [ ]:
df = df.drop('imp_sex', axis=1)

df['date'] = pd.to_datetime(df['dob_yy'].astype(str) + df['dob_wk'].astype(str) + '1', format='%G%V%u')

# Fourier terms for time of day as a type of "seasonality"
df['time_str'] = df['dob_tt'].astype(str).str.zfill(4)
df['hour'] = df['time_str'].str[:2].astype(int)
df['minute'] = df['time_str'].str[2:].astype(int)

df['minute_of_day'] = df['hour'] * 60 + df['minute']
df['time_sin'] = np.sin(2 * np.pi * df['minute_of_day'] / 1440)
df['time_cos'] = np.cos(2 * np.pi * df['minute_of_day'] / 1440)

df = df.drop(['dob_yy', 'dob_tt', 'dob_wk', 'time_str', 'minute_of_day', 'minute', 'hour'], axis=1)

df['sex'] = np.where(df['sex'] == 'M', 1, 0)

In [ ]:
# clean outcome and drop where we don't have a label (we aren't at risk of data-shortage lol)

df = df.rename(columns={'no_mmorb': 'morbidity_reported'})

df = df[df['morbidity_reported'] != 9]
# flipping the binary so morbidity is the positive class
df['morbidity_reported'] = 1 - df['morbidity_reported']

List of variables that require some cleaning in their encodings - lots of sentinel values.

- fagecomb - 99 is a sentinel value; will need to drop or fill it
- priorlive - 99 is a sentinel value
- priordead - 99 sentinel value
- priorterm - 99
- *illb_r (Interval Since Last Live Birth Recode) - 000–003 are sentinel; 888 - Not applicable — first live birth; 999 - Unknown or not stated
- ilop_r (Interval Since Last Other Pregnancy Recode) - not sure if we want to use; same sentinel as illb_r
- *ilp_r (Interval Since Last Pregnancy Recode) - not sure we want to use; same sentinel as illb_r
- ilp_r11 (Interval Since Last Pregnancy Recode 11) - same as above
- precare - 99
- previs - 99
- cig_0 - cig_3 - 99
- bmi - 99.9
- pwgt_r - 999
- dwgt_r - 999
- wtgain - 99
- rf_cesarn - 99
- combgest - 99
- dbwt - 999


In [ ]:
cols_99 = [
    'fagecomb', 'priorlive', 'priordead', 'priorterm',
    'precare', 'previs', 'cig_1', 'cig_2', 'cig_3',
    'wtgain', 'rf_cesarn', 'combgest'
]
df[cols_99] = df[cols_99].replace(99, np.nan)

df[['pwgt_r', 'dwgt_r', 'dbwt']] = df[['pwgt_r', 'dwgt_r', 'dbwt']].replace([999, 9999], np.nan)

df['bmi'] = df['bmi'].replace(99.9, np.nan)
df.loc[df['bmi'] > 90, 'bmi'] = np.nan

df[['illb_r', 'ilp_r']] = df[['illb_r', 'ilp_r']].replace(
    [0, 1, 2, 3, 888, 999],
    np.nan
)

df[['ca_down', 'ca_disor']] = df[['ca_down', 'ca_disor']].replace('C', 2).astype(int)
binary_cols.append('ca_down')

# These seem redundant, so removed for now - can always comment out.
df = df.drop(columns=['ilop_r', 'ilp_r11'], axis=1)

In [ ]:
continuous_cols = [ # 'illb_r', 'ilp_r' <- dropped later on due to too many missing values
    'bmi', 'time_sin', 'time_cos' #, 'month_sin', 'month_cos'
]

discrete_cols = [
    'cig_0', 'cig_1', 'cig_2', 'cig_3', 'priorlive', 'priordead', 'priorterm',
    "precare", 'previs', 'pwgt_r', 'wtgain', 'rf_cesarn', 'combgest', 'dbwt',
    'mager', 'fagecomb'
]

cat_cols = [
    'bfacil', 'mracehisp', 'mar_p', 'meduc', 'fracehisp', 'feduc',
    'rf_pdiab', 'rf_gdiab', 'rf_phype', 'rf_ghype', 'cig_rec',
    'rf_ehype', 'rf_ppterm', 'rf_inftr', 'rf_fedrg', 'rf_artec',
    'rf_cesar', 'ip_gon', 'ip_syph', 'ip_chlam', 'ip_hepb', 'ip_hepc',
    'ld_indl', 'ld_augm', 'ld_ster', 'ld_antb', 'ld_chor', 'ld_anes',
    'me_pres', 'me_rout', 'me_trial', 'mm_mtr', 'mm_plac', 'mm_rupt',
    'mm_uhyst', 'mm_aicu', 'attend', 'pay', 'dplural',
    'ab_aven1', 'ab_aven6', 'ab_nicu', 'ab_surf', 'ab_anti', 'ab_seiz',
    'ca_down', 'ca_disor', 'dob_mm' # using monthly seasonality as one-hot instead of Fourier. Will be more descriptive.
]

In [ ]:
missing_df = (
    df.isna()
      .sum()
      .sort_values(ascending=False)
      .reset_index()
      .set_axis(['variable', 'missing'], axis=1)
)
missing_df['missing_pct'] = missing_df['missing'] / len(df)

missing_df

Drop features that have a count of missing values above a specific threshold. Impute based on the median of the data points' given year of measurement otherwise.

In [ ]:
threshold = 0.20

df = df.drop(columns=missing_df[missing_df['missing_pct'] > threshold]['variable'], axis=1)

for var in missing_df['variable']:
    if var not in df.columns:
        continue

    missing_pct = missing_df.loc[missing_df['variable'] == var, 'missing_pct'].values[0]

    if missing_pct > 0 and df[var].dtype in ['float64']:
        df[var] = (
            df.groupby(df['date'].dt.year)[var]
              .transform(lambda x: x.fillna(x.median()))
        )

In [ ]:
missing_df = (
    df.isna()
      .sum()
      .sort_values(ascending=False)
      .reset_index()
      .set_axis(['variable', 'missing'], axis=1)
)
missing_df['missing_pct'] = missing_df['missing'] / len(df)

missing_df

Some columns are also extremely zero-inflated, so let's create seprate binary versions to encode both y/n and "magnitude". We can select / prune later on.

In [ ]:
df["cig_0_binary"] = (df["cig_0"] > 0).astype(int)

df["cig_1_binary"] = (df["cig_1"] > 0).astype(int)

df["cig_2_binary"] = (df["cig_2"] > 0).astype(int)

df["cig_3_binary"] = (df["cig_3"] > 0).astype(int)

#df["priorlive_binary"] = (df["priorlive"] > 0).astype(int)

df["prior_dead_term_binary"] = ((df["priordead"] > 0) | (df["priorterm"] > 0)).astype(int)

df["precare_binary"] = (df["precare"] > 0).astype(int)

binary_cols.extend(["cig_0_binary", "cig_1_binary", "cig_2_binary", "cig_3_binary", "precare_binary"])
continuous_cols.append('prior_dead_term_binary')

to_remove = ['cig_0', 'cig_1', 'cig_2', 'cig_3', 'priorlive', 'priordead', 'priorterm']
discrete_cols = [c for c in discrete_cols if c not in to_remove]
cat_cols = [c for c in cat_cols if c not in to_remove]

df.drop(to_remove, axis=1, inplace=True)

In [ ]:
list(binary_cols)

In [ ]:
export_df = df[['date', 'morbidity_reported'] + binary_cols + cat_cols + continuous_cols + discrete_cols].copy()


In [ ]:
export_df.head()

In [ ]:
congenital_anomalies = ['ca_anen', 'ca_mnsb', 'ca_cchd', 'ca_cdh', 'ca_omph', 'ca_gast', 'ca_cleft', 'ca_clpal', 'ca_hypo', 'ca_limb']

export_df["congenital_anomalies"] = (
    export_df.loc[:, congenital_anomalies].eq(1).any(axis=1).astype(int)
)

export_df = export_df.drop(columns=congenital_anomalies)

binary_cols = [c for c in binary_cols if c not in congenital_anomalies]
binary_cols.append("congenital_anomalies")

In [ ]:
smoking = ['cig_0_binary', 'cig_1_binary', 'cig_2_binary', 'cig_3_binary', 'cig_rec']

export_df["smoking"] = (
    export_df.loc[:, smoking].eq(1).any(axis=1).astype(int)
)
export_df = export_df.drop(columns=smoking)

binary_cols = [c for c in binary_cols if c not in smoking]
binary_cols.append("smoking")

In [ ]:
# Merge home birth
export_df["hospital_birth_binary"] = export_df["bfacil"].eq(1).astype(int)
export_df = export_df.drop(columns=["bfacil"])

cat_cols.remove("bfacil")
binary_cols.append("bfacil")

In [ ]:
export_df['ca_disor'].value_counts(normalize=True)

In [ ]:
export_df = export_df[(export_df['mracehisp'] != 8) & (export_df['fracehisp'] != 8)]
export_df = export_df[(export_df['meduc'] != 9) & (export_df['feduc'] != 9)]
export_df = export_df[export_df['rf_pdiab'] != 2]
export_df = export_df[export_df['rf_gdiab'] != 2]
export_df = export_df[export_df['rf_phype'] != 2]
export_df = export_df[export_df['rf_ghype'] != 2]
export_df = export_df[export_df['rf_ehype'] != 2]
export_df = export_df[export_df['rf_ppterm'] != 2]
export_df = export_df[export_df['ld_indl'] != 2]
export_df = export_df[export_df['ld_augm'] != 2]
export_df = export_df[export_df['ld_ster'] != 2]
export_df = export_df[export_df['ld_antb'] != 2]
export_df = export_df[export_df['ld_anes'] != 2]
export_df = export_df[export_df['ab_aven1'] != 2]
export_df = export_df[export_df['ab_aven6'] != 2]
export_df = export_df[export_df['ab_nicu'] != 2]
export_df = export_df[export_df['ab_anti'] != 2]

# 1 = y, 2 = no, 3 = unknown, 0 = Not Applicable
# Gonna remove, as I don't know this belongs in
# a causal graph.
export_df = export_df.drop(columns=['mar_p'])

# 8 is other, 9 is unknown
export_df = export_df[~export_df['pay'].isin([8, 9])]

# 3 is other, 9 is unknown
export_df = export_df[~export_df['me_pres'].isin([3, 9])]

# 5 is other, 9 is unknown
export_df = export_df[~export_df['attend'].isin([5, 9])]

# 9 is unknown; 2 is forceps and pretty sparse
# Will drop it if needed
# export_df = export_df[~export_df['me_rout'].isin([9])]
export_df = export_df.drop('me_rout', axis=1)

# 1 & 2 are pretty rare; we'll drop whole column if needed
# export_df = export_df[export_df['ld_chor'] != 2]
export_df = export_df.drop('ld_chor', axis=1)

# This includes rf_fedrg and rf_artec; breaking out
export_df = export_df.drop('rf_inftr', axis=1)

# We have a continuous column (rf_cesarn) for the number of c-sections.
# We can swap if needed
export_df = export_df.drop('rf_cesar', axis=1)

# Combined into one since they're all sparse otherwise
stds = ['ip_gon', 'ip_syph', 'ip_chlam', 'ip_hepb', 'ip_hepc']
export_df['std_pos'] = (
    export_df.loc[:, stds].eq(1).any(axis=1).astype(int)
)

export_df = export_df.drop(columns=stds)

# me_trial - attempts before c-section; might be too correlated with c-section y/n
export_df = export_df.drop('me_trial', axis=1)

# These are actually a part of the target; they're morbidities
export_df = export_df.drop(columns=['mm_mtr', 'mm_plac', 'mm_rupt'], axis=1)

# Merging twins + together
export_df['twins_plus'] = np.where(export_df['dplural'] > 1, 1, 0)

# pretty rare; can drop later if needed
export_df = export_df.drop('ab_surf', axis=1)
# export_df = export_df[export_df['ab_surf'] != 2]

# exceedingly rare
export_df = export_df.drop('ab_seiz', axis=1)

cat_to_binary = [
    'rf_pdiab', 'rf_gdiab', 'rf_phype', 'rf_ghype', 'rf_ehype',
    'rf_ppterm', 'ld_indl', 'ld_augm', 'ld_ster', 'ld_antb', 'ld_anes',
    'ab_aven1', 'ab_aven6', 'ab_nicu', 'ab_anti', 'twins_plus', 'ab_surf'
]

cols_to_remove = [
    'ab_seiz', 'mar_p', 'rf_inftr', 'rf_cesar',
    'mm_mtr', 'mm_plac', 'mm_rupt', 'dplural',
    'me_trial', 'me_rout', 'ld_chor'
]

discrete_cols = [c for c in discrete_cols if c not in cols_to_remove]
cat_cols = [c for c in cat_cols if c not in cols_to_remove]
continuous_cols = [c for c in continuous_cols if c not in cols_to_remove]
binary_cols.extend(cat_to_binary)

In [ ]:
export_df

In [ ]:
export_df['precare'].value_counts(normalize=True)

In [ ]:
export_df.to_csv(BASE_DIR / 'natality_7yr_test_data_for_dag_1.csv', index=False)